# CPC Logging Validation

This notebook contains code associated with validating and analyzing CPC/Raspi logging associated with the sensor box prior to deployment in Revere in March 2021. I will start by just loading and exploring the datasets, and then I will explore the extent of the accuracy and reliability associated with the raspberry pi's data logging.

In [1]:
import bisect
import datetime as dt
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

First, parse the data recorded by the CPC itself.

In [5]:
cpc = pd.read_csv('data/cpc-logged-data.txt', sep='\t');
cpc.describe()


,aveconc,concent,rawconc,cnt_sec,condtmp,satttmp,satbtmp,optctmp,inlttmp,smpflow,...,condpwr,sattpwr,satbpwr,optcpwr,satfpwr,exhfpwr,fillcnt,err_num,mcpcpmp,mcpcpwr
count,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,91185.000000,...,91184.000000,91184.000000,91184.000000,91184.000000,91184.000000,91184.0,91184.000000,91184.0,91184.0,91184.0
mean,637.529407,637.609400,630.513211,3559.149597,21.978993,47.421997,46.996892,32.098040,31.742437,338.022361,...,249.318422,0.514443,66.057740,17.678705,43.170622,200.0,0.002358,0.0,1.0,1.0
std,1227.801271,1227.686741,1208.463286,6823.936140,2.057860,2.187245,2.247951,2.092854,2.238682,1.856988,...,9.104697,9.825600,28.358681,16.470893,1.318105,0.0,0.057406,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,18.400000,18.800000,18.800000,19.100000,19.200000,71.000000,...,0.000000,0.000000,0.000000,0.000000,29.000000,200.0,0.000000,0.0,1.0,1.0
25%,0.000000,0.530000,0.530000,3.000000,20.600000,46.000000,45.600000,30.700000,30.500000,337.000000,...,250.000000,0.000000,45.000000,0.000000,42.000000,200.0,0.000000,0.0,1.0,1.0
50%,99.000000,99.870000,99.870000,562.000000,21.300000,46.800000,46.400000,31.400000,31.300000,338.000000,...,250.000000,0.000000,45.000000,26.000000,43.000000,200.0,0.000000,0.0,1.0,1.0
75%,506.000000,505.000000,505.000000,2854.000000,22.800000,48.300000,47.900000,33.000000,32.800000,339.000000,...,250.000000,0.000000,91.000000,26.000000,44.000000,200.0,0.000000,0.0,1.0,1.0
max,5730.000000,5730.000000,5604.000000,31177.000000,28.800000,54.400000,53.900000,38.900000,37.700000,427.000000,...,250.000000,200.000000,200.000000,200.000000,73.000000,200.0,3.000000,0.0,1.0,1.0


Next, parse the data collected on the raspi.

In [3]:
raspi = pd.read_csv('data/raspi-logged-data.csv');
raspi.describe()

,concent,rawconc,cnt_sec,condtmp,satttmp,satbtmp,optctmp,inlttmp,smpflow,satflow,...,condpwr,sattpwr,satbpwr,optcpwr,satfpwr,exhfpwr,fillcnt,err_num,mcpcpmp,mcpcpwr
count,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,54272.000000,...,54272.000000,54272.0,54272.000000,54272.000000,54272.000000,54272.0,54272.000000,54272.0,0.0,0.0
mean,832.132866,822.539761,4643.132812,20.970677,46.456383,46.026273,31.088816,30.818142,338.060455,338.109393,...,249.602189,0.0,65.569926,17.942770,42.597564,200.0,0.002248,0.0,NaN,NaN
std,1374.440554,1352.582235,7638.008854,0.776005,0.760655,0.778519,0.795496,1.029014,1.531202,1.637550,...,0.820690,0.0,26.013666,15.011424,0.531678,0.0,0.053222,0.0,NaN,NaN
min,0.000000,0.000000,0.000000,19.800000,45.300000,44.700000,29.800000,28.600000,318.000000,329.000000,...,247.000000,0.0,0.000000,0.000000,40.000000,200.0,0.000000,0.0,NaN,NaN
25%,0.880000,0.880000,5.000000,20.100000,45.700000,45.200000,30.300000,30.100000,337.000000,337.000000,...,250.000000,0.0,45.000000,0.000000,42.000000,200.0,0.000000,0.0,NaN,NaN
50%,219.100000,219.100000,1234.000000,21.000000,46.500000,46.100000,31.200000,31.000000,338.000000,338.000000,...,250.000000,0.0,45.000000,26.000000,43.000000,200.0,0.000000,0.0,NaN,NaN
75%,738.800000,738.800000,4161.000000,21.500000,47.000000,46.600000,31.600000,31.500000,339.000000,340.000000,...,250.000000,0.0,91.000000,26.000000,43.000000,200.0,0.000000,0.0,NaN,NaN
max,5078.000000,4979.000000,28193.000000,22.900000,48.300000,48.000000,33.100000,33.000000,347.000000,343.000000,...,250.000000,0.0,200.000000,200.000000,45.000000,200.0,2.000000,0.0,NaN,NaN


# Pre-Processing
I'll start by converting all of the timestamps in each of the files to indexed datetime objects.

In [50]:
# For CPC data.
index = []
for date, time in zip(cpc['#YY/MM/DD'], cpc['HR:MN:SC']):
    Y = int(date[:2]) + 2000
    M = int(date[3:5])
    D = int(date[6:8])
    H = int(time[:2])
    T = int(time[3:5])
    S = int(time[6:8])
    index.append(dt.datetime(Y, M, D, H, T, S))
cpc.index = index

In [51]:
# For Raspi Data
index = []
for timestamp in raspi['#YY/MM/DD:HR:MN:SC']:
    Y = int(timestamp[:4])
    M = int(timestamp[5:7])
    D = int(timestamp[8:10])
    H = int(timestamp[11:13])
    T = int(timestamp[14:16])
    S = int(timestamp[17:19]) # Truncates value instead of rounding
    index.append(dt.datetime(Y, M, D, H, T, S))
raspi.index = index

# Timestamp Observations
TODO: 
1. Confirm that the CPC is always 1 Hz the whole time 
2. See how off the raspi is as an overall chunk of time 
3. See how off the raspi is in terms of frequency
4. Characterize the ways that are off in ways that are relevant to implementation

# Data Validation
TODO 
1. Validate that data at specific timestamps matches on both logs
4. Characterize the ways that are off in ways that are relevant to implementation

In [1]:
cpc.index
raspi.index

NameError: name 'cpc' is not defined